In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi
# fugi = reload(fugi)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import re

#for the cartesian product
import itertools
from itertools import product

#to smooth the MFPADs
import scipy as sp
from scipy import interpolate
import scipy.ndimage
from scipy.spatial import Delaunay
from scipy.ndimage import gaussian_filter

#to create the mesh
import triangulation as tr
from pyntcloud import PyntCloud, structures

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
# import uproot_methods
# [modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

###550eV - 11.5eV CH11
#update nov Kilian binning (36,18)
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CR_Knov_test.root");en="550eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CL_Knov_test.root")
# fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov_test.root")
# fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CL_Knov_test.root")

fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH9_550eV_CR_Knov.root");en="550eV";CH="CH9";
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH9_550eV_CL_Knov.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH9_550eV_CR_Knov.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH9_550eV_CL_Knov.root")

#Select between CH9 and CH11
path = "angular_distr_el/"+CH+"/"

full_72=False

#move first the ID you need
IDs = ["ID_ALL_mol_e0_valid/EN_gate/","ID1_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/"]
dirs = ["MFPADs_multinew/","MFPADs_multinew_phi/"]

loc=path+IDs[2]+dirs[0]
ID=str(IDs[2]).split("_")[0]

MFPAD=False
#load theoretical data
pathth = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=11.5eV';en="KE=11.5eV";

# glob has to be sorted (both in unix and windows)
all_files = sorted(glob.glob(pathth + "/**.dat"))

li = []
cosphi_th = []
test_opening = []
allnames=[]
colnames = ["phi", "cos(theta)", "value"]

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/,
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None)  # r"\s+" is a regex (regular expression)
    # or delim_whitespace=True, it is faste
    temp_name=(filename.replace(" ","")).split("/")[-1].split(".")[0] # adding a column to df with the file name with no spaces and extension
    cosphi_th.append((int(temp_name.split("_")[0]), int(temp_name.split("_")[1])))
    allnames.append(temp_name)
    df["filename"]=temp_name
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

cosphi_th = np.array(cosphi_th)
cosphi_th_adj = np.around(np.array(list(itertools.product(np.linspace(1.,-1.,6).tolist(),np.linspace(-180,180,12).tolist()))),3)
cosphi_th_adj_exp = np.around(np.array(list(itertools.product(np.linspace(0.835,-0.835,6).tolist(),np.linspace(-165,165,12).tolist()))),3)

frame = pd.concat(li, axis=0)
#this commmand avoids 1_1 followed by 1_10: semantic order
frame_srt = frame.groupby("filename",sort=False)
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
#Loading of the experimental MFPADS
MFPAD_RCR,ctheta_RCR,ctheta_RCR_err,cosphi_photon,MFPAD_axis,ctheta_axis,ctheta_axis_red=fugi.import_MFPAD(fileRCR, loc, full=True)
MFPAD_RCL,ctheta_RCL,ctheta_RCL_err=fugi.import_MFPAD(fileRCL, loc)
MFPAD_SCR,ctheta_SCR,ctheta_SCR_err=fugi.import_MFPAD(fileSCR, loc)
MFPAD_SCL,ctheta_SCL,ctheta_SCL_err=fugi.import_MFPAD(fileSCL, loc)

MFPAD_RCR_phi=fugi.sorting_array(MFPAD_RCR, theory=False, a=2)
MFPAD_RCL_phi=fugi.sorting_array(MFPAD_RCL, theory=False, a=2)
MFPAD_SCR_phi=fugi.sorting_array(MFPAD_SCR, theory=False, a=2)
MFPAD_SCL_phi=fugi.sorting_array(MFPAD_SCL, theory=False, a=2)

# MFPAD_RCR_norm=fugi.normalise_with_err(MFPAD_RCR_phi,normtype=2)
# MFPAD_RCL_norm=fugi.normalise_with_err(MFPAD_RCL_phi,normtype=2)
# MFPAD_SCR_norm=fugi.normalise_with_err(MFPAD_SCR_phi,normtype=2)
# MFPAD_SCL_norm=fugi.normalise_with_err(MFPAD_SCL_phi,normtype=2)

#COORDIANTES PHOTON ROTATION
#adjusting cosphi with a shifting to have it equal to theory
ctheta_axis_red=ctheta_axis_red[0]; #Ffrom ccos(theta)

phiM=(MFPAD_axis[0][0][1:] + MFPAD_axis[0][0][:-1])/2 #phi = (36,), MFPAD_axis = (37,)
cosM=(MFPAD_axis[0][1][1:] + MFPAD_axis[0][1][:-1])/2 #COS = (18,), MFPAD_axis = (19,)
phiMM, cosMM = np.meshgrid(phiM, cosM)

phicos_phi=list(product(phiM, cosM))
phi_full = np.array([el[0] for el in phicos_phi])
cos_full = np.array([el[1] for el in phicos_phi])

phi_rad= phi_full*np.pi/180.
theta_rad= np.arccos(cos_full)

print("MFPAD_xy shape = ", MFPAD_axis[0][0].shape)
print("len phi_rad = ", phi_rad.shape)
print("len theta_rad = ", theta_rad.shape)
print("phiM shape = ", phiM.shape)
print("cosM shape = ", cosM.shape)

In [ ]:
#smart flipping for the single 72 MFPAD image. Has to be in phi because of the structure of reshape, do the cos just as a check
temp_RCLCR_phi=np.empty([12,6,648])
temp_SCLCR_phi=np.empty([12,6,648])
temp_totordered_phi_nocos=np.empty([12,6,648])
temp_totordered_phi=np.empty([12,6,648]) #it should be the right sum with opposite cos(theta)_photon

nphi=MFPAD_RCL.shape[1];ncos=MFPAD_RCL.shape[2]

#sum helicities: -cos(theta), phi+180 in photon coordiantes
for j in range(12):
    for i in range(6):
        if j>5:
            temp_RCLCR_phi[j,i]=np.add(MFPAD_RCL_phi.reshape(12,6,-1)[j,i],MFPAD_RCR_phi.reshape(12,6,-1)[j-6,5-i])
            temp_SCLCR_phi[j,i]=np.add(MFPAD_SCL_phi.reshape(12,6,-1)[j,i],MFPAD_SCR_phi.reshape(12,6,-1)[j-6,5-i])

        else:
            temp_RCLCR_phi[j,i]=np.add(MFPAD_RCL_phi.reshape(12,6,-1)[j,i],MFPAD_RCR_phi.reshape(12,6,-1)[6+j,5-i])
            temp_SCLCR_phi[j,i]=np.add(MFPAD_SCL_phi.reshape(12,6,-1)[j,i],MFPAD_SCR_phi.reshape(12,6,-1)[6+j,5-i])

#now they are scanned along phi
MFPAD_RCLCR_phi=np.array(temp_RCLCR_phi).reshape(72,nphi,ncos)
MFPAD_SCLCR_phi=np.array(temp_SCLCR_phi).reshape(72,nphi,ncos)
# MFPAD_RCLCR_phi_norm=fugi.normalise_with_err(MFPAD_RCLCR_phi,normtype=2)
MFPAD_SCLCR_phi_norm=fugi.normalise_with_err(MFPAD_SCLCR_phi,normtype=2)

MFPAD_RCLCR_sum_phi=np.sum(MFPAD_RCLCR_phi, axis=0)
MFPAD_SCLCR_sum_phi=np.sum(MFPAD_SCLCR_phi, axis=0)

#sorting
# MFPAD_RCLCR_cos=fugi.sorting_array(MFPAD_RCLCR, theory=False, a=1)
# MFPAD_RCLCR_phi=fugi.sorting_array(MFPAD_RCLCR, theory=False, a=2) #equal to the original!
# MFPAD_SCLCR_cos=fugi.sorting_array(MFPAD_SCLCR, theory=False, a=1)
# MFPAD_SCLCR_phi=fugi.sorting_array(MFPAD_SCLCR, theory=False, a=2) #equal to the original!
    
temp1=[]
for el in MFPAD_RCLCR_phi:
    temp1.append(np.flip(el,axis=1))
MFPAD_RCLCR_phi_flip=np.array(temp1).reshape(72,nphi,ncos)

tempf=[]
for el in temp_RCLCR_phi.reshape(72,nphi,ncos):
    tempf.append(np.flip(el,axis=1))
temp_RCLCR_phi_flip=np.array(tempf).reshape(12,6,-1)

#sum enantiomer: -cos(theta) fliped axis=1 cos!, -cos(theta) photon coordinates
for j in range(12):
    for i in range(6):
        temp_totordered_phi_nocos[j,i]=np.add(temp_SCLCR_phi[j,i],temp_RCLCR_phi_flip[j,i])
        temp_totordered_phi[j,i]=np.add(temp_SCLCR_phi[j,i],temp_RCLCR_phi_flip[j,5-i])

MFPAD_totordered_phi=np.array(temp_totordered_phi).reshape(72,nphi,ncos)

MFPAD_totordered_phi_norm=fugi.normalise_with_err(MFPAD_totordered_phi,normtype=2)

MFPAD_tot_phi_norm=fugi.normalise_with_err(np.sum(MFPAD_totordered_phi, axis=0),normtype=3)
MFPAD_tot_phi_norm2=fugi.normalise_with_err(np.sum(MFPAD_totordered_phi, axis=0),normtype=2)

#sorting
# MFPAD_totordered_cos=fugi.sorting_array(MFPAD_totordered, theory=False, a=1)
# MFPAD_totordered_phi=fugi.sorting_array(MFPAD_totordered, theory=False, a=2) #equal to the original!

In [ ]:
plt.pcolormesh(MFPAD_SCLCR_sum_phi.T, cmap="viridis")

In [ ]:
plt.pcolormesh(MFPAD_RCLCR_sum_phi.T,cmap="viridis")

In [ ]:
plt.pcolormesh(MFPAD_tot_phi_norm.T, cmap="viridis")

fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_SCLCR_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_RCLCR_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

In [ ]:
if MFPAD:
    #SMART SUM of S enantiomers two different helicities EXPERIMENTAL
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_totordered_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
        ax.tick_params(labelsize=12)
        # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
    fugi.overlaygraph(fig)

In [ ]:
#pick the right theoretical MFPAD and inver the cos(theta)_el axis
#NOTE here the cos(theta)_el is sorted in the range [-1:-1]

counts_temp_cosord=[]
for group in frame_srt:
    counts_temp_cosord.append(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["value"].values)
    if group[0] == "1_1":
        phi_full_sorted=np.array(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["phi"].values)
        ctheta_full_sorted=np.array(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["cos(theta)"].values)

counts_temp_cosord=np.array(counts_temp_cosord).reshape(72,200,100)

#NOTE these will be sorted according to cos(theta)_el [-1:1]
phiM_th=np.unique(phi_full_sorted)
cosM_th=np.unique(ctheta_full_sorted)
xgo_phi,ygo_phi,cosphi_adj_phi = fugi.create_gocoords(a=0, reduced=False, source=True)


xgo_cos_flip,ygo_cos_flip,cosphi_adj_cos_flip = fugi.create_gocoords(a=2, reduced=False, source=True)
dfind=pd.DataFrame(cosphi_adj_cos_flip, columns=["ctheta","phi"])
dfind["item"]=allnames
cosphi_th_std=(dfind.sort_values(by=["ctheta", "phi"], ascending=[False,True]))["item"].values

#NOTE these will be sorted according to cos(theta) or phi PHOTON
# MFPAD_Sth_cos=fugi.sorting_array(counts_temp_cosord, theory=True, items=allnames, a=1)

#MFPAD_S_phi: in photon_coordiantes as allnames 
# OUTPUT a=2 : 6_1', '5_1', '4_1', '3_1', '2_1', '1_1', '6_2', '5_2'
MFPAD_Sth_phi=fugi.sorting_array(counts_temp_cosord, theory=True, items=allnames, a=2)
MFPAD_Stot_phi=np.sum(MFPAD_Sth_phi, axis=0)

print("MFPAD th shape ", MFPAD_Sth_phi.shape)
print("phi_full_sorted shape ", phi_full_sorted.shape)


In [ ]:
if MFPAD:
    #SUM of S enantiomers two different helicities THEORETICAL
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        cs,ax=fugi.plot_interpolation(phiM_th,cosM_th,MFPAD_Sth_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
        ax.tick_params(labelsize=12)
        # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
    fugi.overlaygraph(fig)

fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(axes.ravel()):
    cs,ax=fugi.plot_interpolation(phiM_th,cosM_th,MFPAD_Sth_cos[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

In [ ]:
#down sample theoretical MFPAD to 18 36
#NOTE the flip in the Spline is to fullfill the request of a monotonic theta
temp=[]
# theta = np.linspace(0,np.pi,18)
# phi = np.linspace(0., 2*np.pi, 36)

if full_72:
    for el in MFPAD_Sth_phi:
        f = interpolate.bisplrep(ctheta_full_sorted, phi_full_sorted, el.reshape(-1), s=0.001)
        temp = interpolate.bisplev(cosM,phiM,f).T    
    MFPAD_Sth_phi_red=np.array(temp).reshape(72,36,18)


f = interpolate.bisplrep(ctheta_full_sorted, phi_full_sorted, MFPAD_Stot_phi.reshape(-1), s=0.001)
MFPAD_Stot_phi_red = interpolate.bisplev(cosM,phiM,f).T

In [ ]:
if MFPAD:
    #SUM of S enantiomers two different helicities THEORETICAL red
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_Sth_phi_red[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
        ax.tick_params(labelsize=12)
        # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
    fugi.overlaygraph(fig)

In [ ]:
MFPAD_check=0
if full_72:
    plt.pcolormesh(MFPAD_Sth_phi_red[MFPAD_check].T, cmap="viridis")

plt.pcolormesh(MFPAD_SCR_norm[MFPAD_check].T, cmap="viridis")


plt.pcolormesh(MFPAD_RCLCR_phi[MFPAD_check].T, cmap="viridis")

plt.pcolormesh(MFPAD_SCLCR_phi[MFPAD_check].T, cmap="viridis")

In [ ]:
if full_72:
    plt.pcolormesh(MFPAD_totordered_phi[MFPAD_check].T, cmap="viridis")

In [ ]:
if full_72:
    plt.pcolormesh(gaussian_filter(MFPAD_totordered_phi[MFPAD_check],sigma=1.1).T, cmap="viridis")

In [ ]:
plt.pcolormesh(gaussian_filter(MFPAD_tot_phi_norm,sigma=0.4).T, cmap="viridis")

In [ ]:
plt.pcolormesh(MFPAD_Stot_phi_red.T, cmap="viridis")

In [ ]:
#describe a set of alpha, beta, gamma angles to rotate the theoretical distribution (would be better to rotate before downsampling)
arr=np.arange(-100,100,5)
arrc=np.arange(-3,3,0.5)
#alpha, beta, gamma for 3D rotation
abg=list(product(arr, arr, arr))

a = np.array([el[0] for el in abg])
b = np.array([el[1] for el in abg])
g = np.array([el[2] for el in abg])

print("len(abg) = ",len(abg))

for i,el in enumerate(abg):
    if el == (1,1,1) or el == (-1,-1,-1) or np.all(np.isclose(el, [0.2, 0.2, 0.2], rtol=1e-05)) or el == (0,0,0):
        zeropoint=i

print(abg[zeropoint])
print(f'at {zeropoint:d}')

In [ ]:
%store MFPAD_Stot_phi_red
%store theta_rad
%store phi_rad
%store phiMM
%store phiM
%store cosMM
%store cosM

In [ ]:
# REDUCED TO 1 MFPAD! rotate the theoretical distribution
fugi = reload(fugi)
r_temp=[];n_temp=[]

r_temp.append(fugi.rot3d_MFPAD_dist(MFPAD_Stot_phi_red,theta_rad,phi_rad,abg,phiM,cosM, convention=1, s=0.1))
MFPAD_Stot_phi_rotated=np.array(r_temp).reshape(1,len(abg),36,18)

n_temp.append(fugi.normalise_with_err(MFPAD_Stot_phi_rotated[0],normtype=3))
MFPAD_Stot_phi_rotated_norm=np.array(n_temp).reshape(1,len(abg),36,18)

In [ ]:
d_temp=[]
for el in MFPAD_Stot_phi_rotated_norm[0]:
    d_temp.append(np.sqrt(np.sum((gaussian_filter(MFPAD_tot_phi_norm,sigma=0.4).reshape(-1)-el.reshape(-1))**2)))

#between 0 and 1
d_single_norm=fugi.normalise_with_err(np.array(d_temp),normtype=2)

print("d_sum shape = ", d_single_norm.shape)

d_single_complete_norm=np.column_stack((a,b,g,d_single_norm)).reshape(-1,4)

d_min_norm=d_single_norm.min()
delta=d_single_norm.max()-d_single_norm.min()

output=[]
for i,el in enumerate(d_single_complete_norm):
    if el[3] <= d_min_norm+delta*0.02:
        output.append(np.append(el,i))
sort_out=sorted(output, key = lambda x:x[3])

print("For the sum MFPAD")
print(sort_out)

In [ ]:
plt.pcolormesh(MFPAD_Stot_phi_rotated[0][zeropoint].T,cmap="viridis")

In [ ]:
plt.pcolormesh(gaussian_filter(MFPAD_tot_phi_norm,sigma=0.4).T, cmap="viridis")

In [ ]:
plt.pcolormesh(MFPAD_Stot_phi_rotated_norm[0][int(sort_out[1][4])].T,cmap="viridis")

In [ ]:
#rotate the theoretical distribution
#fugi = reload(fugi)
tot_shape=MFPAD_Stot_phi_rotated_norm.shape[0]

if full_72:
    #for 72 MFPADS the calculation can last for 15 minutes for (-30,34,4)
    r_temp=[];n_temp=[]

    # these ae now 72 x len(arr) MFPAD (each 36,18 = 648)
    for el in MFPAD_Sth_phi_red:
        r_temp.append(fugi.rot3d_MFPAD_dist(el,theta_rad,phi_rad,abg,phiMM,cosMM, convention=1))

    MFPAD_Sth_phi_rotated=np.array(r_temp).reshape(72,len(abg),36,18)

    for el in MFPAD_Sth_phi_rotated:
        n_temp.append(fugi.normalise_with_err(el,normtype=2))

    MFPAD_Sth_phi_rotated_norm=np.array(n_temp).reshape(72,len(abg),36,18)

    plt.pcolormesh(gaussian_filter(MFPAD_Sth_phi_rotated_norm[71][zeropoint].reshape(36,18),sigma=0.6).T, cmap="viridis")


    #difference square looping along all coordiantes: NOTE they should have filled int he same way
    #l2 norm
    s_temp=[];
    # for i,el_exp in enumerate(MFPAD_SCL_norm): #one enantiomer, one helicity
    for i,el_exp in enumerate(MFPAD_totordered_phi_norm): #all together
        d_temp=[]
        if i < tot_shape:
            for j,el_th in enumerate(MFPAD_Sth_phi_rotated_norm[i]):
                if i == 0 and j == zeropoint: #have a look fot the zero point few cells before!!
                    test_0=gaussian_filter(el_exp,0.6).reshape(-1) - gaussian_filter(el_th,0.5).reshape(-1)
                    plt.imshpcolormeshow(np.sqrt(test_0.reshape(36,18)**2).T, cmap=cmap_temp)
                    plt.colorbar()
                elif i == 71 and j == zeropoint: #have a look fot the zero point few cells before!!
                    test_71=gaussian_filter(el_exp,0.6).reshape(-1) - gaussian_filter(el_th,0.5).reshape(-1)
                    plt.pcolormesh(np.sqrt(test_71.reshape(36,18)**2).T, cmap=cmap_temp)
                    plt.colorbar()
                d_temp.append(np.sum((gaussian_filter(el_exp,sigma=0.6).reshape(-1)-gaussian_filter(el_th,sigma=0.5).reshape(-1))**2)) #len(d_temp) = len(abg)
            s_temp.append(np.sqrt(d_temp)) #len(s_temp) = len(d_temp)*72
        else:
            break

    d_all=np.array(s_temp).reshape(tot_shape,len(abg))
    d_sum=np.sum(d_all, axis=0)
    d_norm=fugi.normalise_with_err(np.array(d_sum),normtype=2)

    print("d_temp length = ", len(d_temp))
    print("s_temp shaoe = ", np.array(s_temp).shape)
    print("d_all shape = ", d_all.shape)
    print("d_sum shape = ", d_sum.shape)

    d_tot=np.column_stack((a,b,g,d_sum)).reshape(-1,4)
    d_tot_norm=np.column_stack((a,b,g,d_norm)).reshape(-1,4)

    d_min=d_sum.min()
    d_min_norm=d_norm.min()
    delta=d_norm.max()-d_norm.min()

    output=[]
    for el in d_tot_norm:
        if el[3] <= d_min_norm+delta*0.05:
            output.append(el)
    print("For the collective 72 MFPADs")
    sorted(output, key = lambda x:x[3])

new=[]
for el in MFPAD_rotated_norm:
    for el1 in el:
        new.append(np.nan_to_num(el1.reshape(-1),nan=np.average(np.nan_to_num(el1.reshape(-1)))))

MFPAD_rotated_norm=new

#it works but it takes HOURS!!
import matplotlib.animation as ani
fig = plt.figure()

def animate(i=int):
    p=plt.pcolormesh(MFPAD_Stot_phi_rotated_norm[0][i].T, cmap="viridis")
    return p

animator = ani.FuncAnimation(fig, animate, frames = len(abg))
animator.save("OUTPUTS/test.mp4", fps =30, writer="ffmpeg")
plt.show()

In [ ]:
if full_72 and tot_shape > 2:
    d_temp_s=[]
    for el_th in MFPAD_Sth_phi_rotated_norm[71]:
        d_temp_s.append(np.sum((gaussian_filter(MFPAD_SCLCR_phi_norm[71],sigma=1.1).reshape(-1)-gaussian_filter(el_th,sigma=0.5).reshape(-1))**2)) #len(d_temp) = len(abg)
        # d_temp_s.append(np.sum((MFPAD_totordered_phi_norm[71].reshape(-1)-el_th.reshape(-1))**2)) #len(d_temp) = len(abg)

    d_all_s=np.sqrt(np.array(d_temp_s))
    d_norm_s=fugi.normalise_with_err(np.array(d_all_s),normtype=2)

    print("d_temp_s length = ", len(d_temp_s))
    print("d_norm_s shape = ", d_norm_s.shape)

    print("For the S enantiomer MFPADs")
    
    d_tot_s=np.column_stack((a,b,g,d_norm_s)).reshape(-1,4)
    d_min_s=d_norm_s.min()
    output_s=[]
    for el in d_tot_s:
        if el[3] <= d_min_s*1.00005:
            output_s.append(el)
    sorted(output_s, key = lambda x:x[3])

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')

# d_masked = np.ma.masked_array(d_norm, d_norm>=d_norm.min()+delta*0.7)
# ax.scatter(a, b, g, c=d_masked, alpha=0.2, s=10, cmap=cmap_temp)

# d_masked = np.ma.masked_array(d_norm, d_norm>=d_norm.min()+delta*0.5)
# img = ax.scatter(a, b, g, c=d_masked, alpha=0.5, s=50, cmap=cmap_temp)

d_masked = np.ma.masked_array(d_single_norm, d_single_norm>=d_min_norm+delta*0.2)
img= ax.scatter(a, b, g, c=d_masked, alpha=0.8, s=20, cmap=cmap_temp)
# ax.scatter(a, b, c=d_masked, zs=-90, zdir='z', s=60, cmap=plt.hot())
# ax.scatter(a, g, c=d_masked, zs=-70, zdir='y', s=60, cmap=plt.hot())
# ax.scatter(b, g, c=d_masked, zs=70,  zdir='x', s=60, cmap=plt.hot())

ax.set_xlabel('x alpha [DEG]', labelpad=25)
ax.set_ylabel('y beta [DEG]', labelpad=25)
ax.set_zlabel('z gamma [DEG]', labelpad=25)

ax.view_init(elev=30., azim=1800.)

fig.colorbar(img)
plt.show()

#if d_norm 
fig1 = go.Figure(data=[go.Scatter3d(x=a, y=b, z=g, mode='markers',
                    marker=dict( size=5,color=d_single_norm, colorscale='temps', opacity=1., showscale=True)
                    #color set colours to an array/list of desired values  
                    #show scale to show the legend according to the color
                    )])

#fig1.update_layout(title="TFMeOx MFPADs EXP", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))

#fig1.add_trace(go.Surface(x=a, y=b, z=g, colorscale="blues",  showscale=False))
##fig1.add_trace(go.Surface(x=x.apply(lambda x: 10), y=y, z = np.array([z]*length_data), colorscale= bright_blue, showscale=False))
##fig1.add_trace(go.Surface(x=x, y= y.apply(lambda x: 30), z =  np.array([z]*length_data).transpose(), colorscale=bright_pink, showscale=False))
fig1.show()

fig1 = go.Figure(data=[go.Scatter3d(x=d_tot_s[:,0], y=d_tot_s[:,1], z=d_tot_s[:,2], mode='markers',
                    marker=dict( size=5,color=d_tot_s[:,3], colorscale='temps', opacity=.8, showscale=True)
                    #color set colours to an array/list of desired values  
                    #show scale to show the legend according to the color
                    )])

#fig1.update_layout(title="TFMeOx MFPADs EXP", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))
fig1.show()

from scipy import interpolate

f_th = interpolate.interp2d(cosM_th,phiM_th,MFPAD_Sth_phi[71], kind='cubic')
f_exp = interpolate.interp2d(cosM,phiM,MFPAD_totordered_phi[71], kind='cubic')

ynew = np.linspace(-180, 180, 36)
xnew = np.linspace(-1, 1, 18)
znew = f_th(xnew,ynew)
plt.pcolormesh(znew.T, cmap="viridis")
plt.show()

from scipy.spatial.transform import Rotation, RotationSpline
q = Rotation.from_euler('ZYX', abg, degrees=True)
spline = RotationSpline(znew.reshape(-1), q)